In [2]:
import json
from bs4 import Tag
from tqdm import tqdm
import tb_xzxw_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [3]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = tb_xzxw_utils.scrape_tb_xzxw_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_tb_xzxw_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"tb_xzxw_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [ ]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tb_xzxw_ALL_link_ཆེད་སྒྲིག.json: 1Total page in tb_xzxw_ALL_link_བརྩམས་ཆོས་ལྟ་ཀློག.json: 4
page key name: Page བརྩམས་ཆོས་ལྟ་ཀློག 4
Page key code: Page བརྩམས་ཆོས་ལྟ་ཀློག 

page key name: Page ཆེད་སྒྲིག 1
Page key code: Page ཆེད་སྒྲིག 
Total page in tb_xzxw_ALL_link_སྲིད་གཞུང་ཡིག་ཆ།.json: 30
page key name: Page སྲིད་གཞུང་ཡིག་ཆ། 30
Page key code: Page སྲིད་གཞུང་ཡིག་ཆ། 
Total page in tb_xzxw_ALL_link_སྲིད་དོན་གསར་འགྱུར།.json: 30
page key name: Page སྲིད་དོན་གསར་འགྱུར། 30
Page key code: Page སྲིད་དོན་གསར་འགྱུར། 
Total page in tb_xzxw_ALL_link_འགོ་ཁྲིད་བྱེད་སྒོ།.json: 30
page key name: Page འགོ་ཁྲིད་བྱེད་སྒོ། 30
Page key code: Page འགོ་ཁྲིད་བྱེད་སྒོ། 
Total page in tb_xzxw_ALL_link_སྲིད་གཞུང་སྤྱི་བསྒྲགས།.json: 30
page key name: Page སྲིད་གཞུང་སྤྱི་བསྒྲགས། 30
Page key code: Page སྲིད་གཞུང་སྤྱི་བསྒྲགས། 


0it [00:00, ?it/s]


Total Failure in the ཆེད་སྒྲིག article: 0
tb_xzxw_ALL_content_ཆེད་སྒྲིག.json
Successfully saved: tb_xzxw_ALL_content_ཆེད་སྒྲིག.json
Total page in tb_xzxw_ALL_link_ཏང་སྲིད་སྒྲིག་གཞི།.json: 17
page key name: Page ཏང་སྲིད་སྒྲིག་གཞི། 17
Page key code: Page ཏང་སྲིད་སྒྲིག་གཞི། 
Total page in tb_xzxw_ALL_link_སྲིད་ཇུས་གསལ་འགྲེལ།.json: 30
page key name: Page སྲིད་ཇུས་གསལ་འགྲེལ། 30
Page key code: Page སྲིད་ཇུས་གསལ་འགྲེལ། 
Total page in tb_xzxw_ALL_link_མི་དོན་བསྐོ་འཐེན།.json: 17
page key name: Page མི་དོན་བསྐོ་འཐེན། 17
Page key code: Page མི་དོན་བསྐོ་འཐེན། 
Total page in tb_xzxw_ALL_link_རྒྱབ་ཤ་དཀར་པོའི་དེབ།.json: 4
page key name: Page རྒྱབ་ཤ་དཀར་པོའི་དེབ། 4
Page key code: Page རྒྱབ་ཤ་དཀར་པོའི་དེབ། 
Total page in tb_xzxw_ALL_link_དམངས་ཕན་སྲིད་ཇུས།.json: 18
page key name: Page དམངས་ཕན་སྲིད་ཇུས། 18
Page key code: Page དམངས་ཕན་སྲིད་ཇུས། 


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]


  0%|          | 0/600 [00:00<?, ?it/s]



  0%|          | 0/339 [00:00<?, ?it/s]




  0%|          | 0/349 [00:00<?, ?it/s]





  0%|          | 0/600 [00:00<?, ?it/s]






  0%|          | 0/325 [00:00<?, ?it/s]







  0%|          | 0/600 [00:00<?, ?it/s]









  0%|          | 0/580 [00:00<?, ?it/s]








  0%|          | 0/584 [00:00<?, ?it/s]

  1%|▏         | 1/78 [00:01<02:04,  1.62s/it]

  3%|▎         | 2/78 [00:01<01:00,  1.26it/s]

  6%|▋         | 5/78 [00:02<00:21,  3.45it/s]

  8%|▊         | 6/78 [00:02<00:26,  2.74it/s]

  9%|▉         | 7/78 [00:02<00:21,  3.26it/s]

 13%|█▎        | 10/78 [00:03<00:12,  5.67it/s]

 14%|█▍        | 11/78 [00:03<00:12,  5.42it/s]

 17%|█▋        | 13/78 [00:04<00:17,  3.61it/s]

 19%|█▉        | 15/78 [00:04<00:13,  4.82it/s]

 21%|██        | 16/78 [00:04<00:12,  5.14it/s]

 24%|██▍       | 19/78 [00:05<00:14,  3.97it/s]

 26%|██▌       | 20/78

Total Failure in the བརྩམས་ཆོས་ལྟ་ཀློག article: 77
tb_xzxw_ALL_content_བརྩམས་ཆོས་ལྟ་ཀློག.json
Successfully saved: tb_xzxw_ALL_content_བརྩམས་ཆོས་ལྟ་ཀློག.json
Total page in tb_xzxw_ALL_link_གྲོང་གསེབ་གསར་པ་འཛུགས་སྐྲུན།.json: 30
page key name: Page གྲོང་གསེབ་གསར་པ་འཛུགས་སྐྲུན། 30
Page key code: Page གྲོང་གསེབ་གསར་པ་འཛུགས་སྐྲུན། 


...



  0%|          | 0/600 [00:00<?, ?it/s]



 50%|█████     | 303/600 [04:32<05:01,  1.02s/it]



 51%|█████     | 305/600 [04:32<03:06,  1.58it/s]






 40%|████      | 240/600 [03:37<03:39,  1.64it/s]



 51%|█████     | 307/600 [04:32<01:58,  2.47it/s]









 34%|███▍      | 187/545 [02:47<06:00,  1.01s/it]



 51%|█████▏    | 308/600 [04:32<01:57,  2.48it/s]







  7%|▋         | 33/480 [00:35<11:18,  1.52s/it]







  7%|▋         | 34/480 [00:35<08:16,  1.11s/it]



 52%|█████▏    | 309/600 [04:33<01:57,  2.47it/s]







  7%|▋         | 35/480 [00:35<06:48,  1.09it/s]







  8%|▊         | 36/480 [00:35<05:03,  1.46it/s]



 52%|█████▏    | 310/600 [04:33<01:59,  2.42it/s]









 34%|███▍      | 188/545 [02:49<06:01,  1.01s/it]








 10%|▉         | 48/500 [00:50<10:10,  1.35s/it]








 10%|█         | 50/500 [00:51<06:02,  1.24it/s]







  8%|▊         | 37/480 [00:36<04:43,  1.56it/s]



 52%|█████▏    | 312/600 [04:34<01:41,  2.83it/s]






 40%|████    

Total Failure in the ལུས་ཁམས་འཕྲོད་སྦྱོར། article: 210
tb_xzxw_ALL_content_ལུས་ཁམས་འཕྲོད་སྦྱོར།.json
Successfully saved: tb_xzxw_ALL_content_ལུས་ཁམས་འཕྲོད་སྦྱོར།.json
Total page in tb_xzxw_ALL_link_བོད་ལྗོངས་སྣོད་བཅུད།.json: 30
page key name: Page བོད་ལྗོངས་སྣོད་བཅུད། 30
Page key code: Page བོད་ལྗོངས་སྣོད་བཅུད། 












  0%|          | 0/600 [00:00<?, ?it/s]






 45%|████▍     | 206/460 [01:21<00:54,  4.62it/s]







 11%|█         | 29/269 [00:28<02:28,  1.61it/s]



 34%|███▍      | 190/558 [01:51<01:11,  5.15it/s]






 45%|████▌     | 207/460 [01:21<00:54,  4.65it/s]



 34%|███▍      | 191/558 [01:51<01:05,  5.59it/s]






 45%|████▌     | 208/460 [01:21<00:51,  4.94it/s]



 34%|███▍      | 192/558 [01:51<01:10,  5.22it/s]



 35%|███▍      | 193/558 [01:51<01:02,  5.84it/s]







 11%|█         | 30/269 [00:29<02:24,  1.65it/s]

 32%|███▏      | 191/600 [02:53<05:30,  1.24it/s]






 46%|████▌     | 210/460 [01:22<00:41,  5.98it/s]








 50%|█████     | 251/500 [03:42<03:53,  1.06it/s]



 35%|███▍      | 194/558 [01:52<01:34,  3.84it/s]








 50%|█████     | 252/500 [03:42<03:06,  1.33it/s]






 46%|████▌     | 212/460 [01:22<01:01,  4.01it/s]



 35%|███▍      | 195/558 [01:52<01:56,  3.12it/s]

 32%|███▏      | 193/600 [02:54<04:39,  1.46it/s]



 35%|███▌      | 19

Total Failure in the མི་སྣ། article: 401
tb_xzxw_ALL_content_མི་སྣ།.json
Successfully saved: tb_xzxw_ALL_content_མི་སྣ།.json
Total page in tb_xzxw_ALL_link_ཁོར་ཡུག་སྲུང་སྐྱོང་།.json: 30
page key name: Page ཁོར་ཡུག་སྲུང་སྐྱོང་། 30
Page key code: Page ཁོར་ཡུག་སྲུང་སྐྱོང་། 


 76%|███████▋  | 382/500 [04:36<00:26,  4.50it/s]






  0%|          | 0/600 [00:00<?, ?it/s]



 78%|███████▊  | 436/558 [02:45<00:39,  3.11it/s]



 79%|███████▊  | 439/558 [02:45<00:24,  4.94it/s]







 43%|████▎     | 116/269 [01:23<00:41,  3.73it/s]









 11%|█         | 64/600 [00:54<06:13,  1.44it/s]








 77%|███████▋  | 383/500 [04:36<00:25,  4.55it/s]









 11%|█         | 66/600 [00:55<04:09,  2.14it/s]



 79%|███████▉  | 441/558 [02:46<00:23,  5.02it/s]







 44%|████▍     | 118/269 [01:23<00:34,  4.36it/s]








 77%|███████▋  | 384/500 [04:36<00:28,  4.04it/s]








 77%|███████▋  | 386/500 [04:36<00:20,  5.45it/s]








 78%|███████▊  | 388/500 [04:36<00:15,  7.12it/s]







 44%|████▍     | 119/269 [01:24<00:38,  3.91it/s]







 45%|████▍     | 120/269 [01:24<00:36,  4.03it/s]



 79%|███████▉  | 443/558 [02:47<00:28,  4.05it/s]









 11%|█         | 67/600 [00:56<05:19,  1.67it/s]








 78%|███████▊  | 389/500 [04:37<00:27,  3.97it/s]


In [ ]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files